In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
import datetime
import time
import ast
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import timedelta

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Final Data csv/Finals/Final.csv', parse_dates=['created_at'])

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
senti = pd.read_csv('/content/drive/My Drive/Final Data csv/Finals/Final_Sentiment_Analysis.csv')

In [ ]:
#to extract user id
def user_id(df, senti):
  df['user_id'] = pd.DataFrame((data.user.str.split(expand=True)[1]).str.split(",",expand=True)[0])
  df['user_id'] = df['user_id'].astype(int)
  df.sort_values('user_id', inplace=True)
  a = df['user_id'].value_counts().index[df['user_id'].value_counts()>=3]
  col = pd.DataFrame({'user_id':a})
  df = df.merge(col, left_on=None, right_on=None)
  df.drop(columns=['Unnamed: 0'], axis=1, inplace=True)
  df = pd.concat([df,senti], axis=1, join='inner')
  df = df[['user_id','id','created_at','full_text','user','retweet_count','sentiment']]
  return df

In [ ]:
data = user_id(data, senti)
ff = data.groupby(by=['user_id','id'])
grouped_data = pd.DataFrame(ff.first())

In [ ]:
data.shape

(70334, 7)

In [ ]:
def avg_time_diff(grouped_data):
  ll = []
  count = 0
  elapsed_time = timedelta(hours=0,minutes=0, seconds=0)
  for i in range(len(grouped_data)):
    if (grouped_data['created_at'].index[i][0]!=grouped_data['created_at'].index[-1][0] and grouped_data['created_at'].index[i][0]==grouped_data['created_at'].index[i+1][0]):
      count = count + 1
      elapsed_time = elapsed_time + (grouped_data['created_at'][i+1] - grouped_data['created_at'][i])
    elif (grouped_data['created_at'].index[i][0]!=grouped_data['created_at'].index[-1][0] and grouped_data['created_at'].index[i][0]!=grouped_data['created_at'].index[i+1][0]):
      ll.append(elapsed_time/(count+1))
      count = 0
      elapsed_time = timedelta(hours=0,minutes=0, seconds=0)
    elif (i==(len(grouped_data.index))-1):
      ll.append(elapsed_time/(count+1))
      break
    elif (grouped_data['created_at'].index[i][0]==grouped_data['created_at'].index[-1][0]):
      count = count + 1
      elapsed_time = elapsed_time + (grouped_data['created_at'][i+1] - grouped_data['created_at'][i])
  return ll

In [ ]:
def followers_count(grouped_data):
  ll = []
  l=[]
  lol=[]
  for i in range(len(grouped_data)):
    if (grouped_data['user'].index[i][0]!=grouped_data['user'].index[-1][0] and grouped_data['user'].index[i][0]==grouped_data['user'].index[i+1][0]):
      res = ast.literal_eval(grouped_data.user[i])
      l = [lol.append(v) for k,v in res.items() if k=='followers_count']
    elif (grouped_data['user'].index[i][0]!=grouped_data['user'].index[-1][0] and grouped_data['user'].index[i][0]!=grouped_data['user'].index[i+1][0]):
      res = ast.literal_eval(grouped_data.user[i])
      l = [lol.append(v) for k,v in res.items() if k=='followers_count']
      lol.sort()
      ll.append(lol[-1])
      lol = []
    elif (i==(len(grouped_data.index))-1):
      res = ast.literal_eval(grouped_data.user[i])
      l = [lol.append(v) for k,v in res.items() if k=='followers_count']
      lol.sort()
      ll.append(lol[-1])
      break
    elif (grouped_data['created_at'].index[i][0]==grouped_data['created_at'].index[-1][0]):
      res = ast.literal_eval(grouped_data.user[i])
      l = [lol.append(v) for k,v in res.items() if k=='followers_count']
  return ll

In [ ]:
def retweet_count(grouped_data):
  ll = []
  l=[]
  for i in range(len(grouped_data)):
    if (grouped_data['retweet_count'].index[i][0]!=grouped_data['retweet_count'].index[-1][0] and  grouped_data['retweet_count'].index[i][0]==grouped_data['retweet_count'].index[i+1][0]):
      l.append((grouped_data['retweet_count'][i]).astype(int))
    elif (grouped_data['retweet_count'].index[i][0]!=grouped_data['retweet_count'].index[-1][0] and grouped_data['retweet_count'].index[i][0]!=grouped_data['retweet_count'].index[i+1][0]):
      l.append((grouped_data['retweet_count'][i]).astype(int))
      l.sort()
      ll.append(l[-1])
      l = []
    elif (i==(len(grouped_data.index))-1):
      l.append((grouped_data['retweet_count'][i]).astype(int))
      l.sort()
      ll.append(l[-1])
      break
    elif (grouped_data['retweet_count'].index[i][0]==grouped_data['retweet_count'].index[-1][0]):
      l.append((grouped_data['retweet_count'][i]).astype(int))
  return ll

In [ ]:
def minutes(time):
    return (time.days*24*60 + time.seconds/60) 

In [ ]:
def hours(time):
    return (time.days*24 + time.seconds/3600) 

In [ ]:
senti = {'positive':1, 'neutral':0, 'negative':-1}
grouped_data['sentiment'] = grouped_data['sentiment'].map(senti)

In [ ]:
def sentiment_score(grouped_data):
  ll = []
  l = []
  v = 0
  for i in range(len(grouped_data)):
    if (grouped_data['sentiment'].index[i][0]!=grouped_data['sentiment'].index[-1][0] and grouped_data['sentiment'].index[i][0]==grouped_data['sentiment'].index[i+1][0]):
      l.append(grouped_data['sentiment'][i].astype(int))
    elif (grouped_data['sentiment'].index[i][0]!=grouped_data['sentiment'].index[-1][0] and grouped_data['sentiment'].index[i][0]!=grouped_data['sentiment'].index[i+1][0]):
      l.append(grouped_data['sentiment'][i].astype(int))
      v = sum(l)
      ll.append(v)
      l = []
      v = 0
    elif (i==(len(grouped_data.index))-1):
      l.append(grouped_data['sentiment'][i].astype(int))
      v = sum(l)
      ll.append(v)
      break
    elif (grouped_data['sentiment'].index[i][0]==grouped_data['sentiment'].index[-1][0]):
      l.append(grouped_data['sentiment'][i].astype(int))
  return ll

In [ ]:
def sentiment(grouped_data):
  ss = []
  for i in range(len(grouped_data)):
    if grouped_data.sentiment_score[i] > 0:
      ss.append('positive')
    elif grouped_data.sentiment_score[i] < 0:
      ss.append('negative')
    else:
      ss.append('neutral')
  return ss

In [ ]:
def extreme_user(df):
  ll = []
  for i in df['avg_time_difference_in_min']:
    if i<=90:
      ll.append(1)
    else:
      ll.append(0)
  return ll

In [ ]:
df = pd.DataFrame()
df['user_id'] = data.user_id.unique()
df['avg_time_difference'] = avg_time_diff(grouped_data)
ll1=[]
ll2=[]
for i in df['avg_time_difference']:
  ll1.append(round(minutes(i),2))
  ll2.append(round(hours(i),3))
df['avg_time_difference_in_min'] = ll1
df['avg_time_difference_in_hours'] = ll2
df['followers_count'] = followers_count(grouped_data)
df['retweet_count'] = retweet_count(grouped_data)
df['sentiment_score'] = sentiment_score(grouped_data)
df['sentiment'] = sentiment(df)
df['extreme_user'] = extreme_user(df)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: overflow encountered in long_scalars
  # Remove the CWD from sys.path while we load stuff.


In [ ]:
df.head()

,user_id,avg_time_difference,avg_time_difference_in_min,avg_time_difference_in_hours,followers_count,retweet_count,sentiment_score,sentiment,extreme_user
0,13508182,16:50:12,1010.20,16.837,147919,1,3,positive,0
1,14458945,01:22:35,82.58,1.376,282,0,4,positive,1
2,16263847,13:11:17,791.28,13.188,2613,1147,2,positive,0
3,16804541,11:39:55.500000,699.92,11.665,12203,9,1,positive,0
4,16863403,13:34:30,814.50,13.575,664,1597,0,neutral,0


In [ ]:
df.to_csv('Best_Ever_Dataset.csv')

In [ ]:
df.shape

(10984, 9)